# Working with Voyage AI in Pixeltable

Pixeltable's Voyage AI integration enables you to access state-of-the-art embedding and reranker models via the Voyage AI API.

### Prerequisites
- A Voyage AI account with an API key (https://www.voyageai.com/)

### Important Notes

- Voyage AI usage may incur costs based on your Voyage AI plan.
- Be mindful of sensitive data and consider security measures when integrating with external services.


First you'll need to install required libraries and enter your Voyage AI API key.


In [ ]:
%pip install -qU pixeltable voyageai

In [ ]:
import os
import getpass
if 'VOYAGEAI_API_KEY' not in os.environ:
    os.environ['VOYAGEAI_API_KEY'] = getpass.getpass('Enter your Voyage AI API key:')


Now let's create a Pixeltable directory to hold the tables for our demo.


In [ ]:
import pixeltable as pxt

# Remove the 'voyageai_demo' directory and its contents, if it exists
pxt.drop_dir('voyageai_demo', force=True)
pxt.create_dir('voyageai_demo')


## Text Embeddings

Voyage AI provides state-of-the-art embedding models for semantic search and RAG applications.


In [ ]:
from pixeltable.functions import voyageai

# Create a table for document embeddings
docs_t = pxt.create_table('voyageai_demo.documents', {'text': pxt.String})

# Add computed column with Voyage embeddings
docs_t.add_computed_column(
    embedding=voyageai.embeddings(
        docs_t.text,
        model='voyage-3.5',
        input_type='document'
    )
)


In [ ]:
# Insert some sample documents
documents = [
    "The Mediterranean diet emphasizes fish, olive oil, and vegetables, believed to reduce chronic diseases.",
    "Photosynthesis in plants converts light energy into glucose and produces essential oxygen.",
    "20th-century innovations, from radios to smartphones, centered on electronic advancements.",
    "Rivers provide water, irrigation, and habitat for aquatic species, vital for ecosystems.",
    "Apple's conference call to discuss fourth fiscal quarter results is scheduled for Thursday, November 2, 2023.",
    "Shakespeare's works, like 'Hamlet' and 'A Midsummer Night's Dream,' endure in literature."
]

docs_t.insert([{'text': doc} for doc in documents])


In [ ]:
# View the embeddings
docs_t.select(docs_t.text, docs_t.embedding).head(3)


## Reranking

Voyage AI's rerankers can refine search results by providing more accurate relevance scores.


In [ ]:
# Create a table for reranking
rerank_t = pxt.create_table(
    'voyageai_demo.rerank',
    {'query': pxt.String, 'documents': pxt.Json}
)

# Add computed column with reranking results
rerank_t.add_computed_column(
    reranked=voyageai.rerank(
        rerank_t.query,
        rerank_t.documents,
        model='rerank-2.5',
        top_k=3
    )
)


In [ ]:
# Insert query and documents to rerank
rerank_t.insert([{
    'query': "When is Apple's conference call scheduled?",
    'documents': documents
}])


In [ ]:
# View reranking results
results = rerank_t.select(rerank_t.query, rerank_t.reranked).collect()
reranked_results = results[0]['reranked']['results']

print("Top 3 most relevant documents:\n")
for i, result in enumerate(reranked_results, 1):
    print(f"{i}. (Score: {result['relevance_score']:.3f})")
    print(f"   {result['document']}\n")


### Learn More

To learn more about RAG operations in Pixeltable, check out the [RAG Operations in Pixeltable](https://docs.pixeltable.com/notebooks/use-cases/rag-operations) tutorial.

For more information about Voyage AI models and features, visit:
- [Voyage AI Documentation](https://docs.voyageai.com/)
- [Text Embeddings](https://docs.voyageai.com/docs/embeddings)
- [Rerankers](https://docs.voyageai.com/docs/reranker)

If you have any questions, don't hesitate to reach out.
